In [1]:
import os
import numpy as np
import tensorflow as tf
import tensorlayer as tl
import tensorlayer.layers as tly
from groupy.gconv.tensorflow_gconv.splitgconv2d import gconv2d, gconv2d_util



In [2]:
def gconv_bn_act(x, gconv_type, C_in, C_out, ksize=3, padding='VALID', 
                 bn=True, is_train=True, act=tf.nn.relu, 
                 name='gconv_bn_act', reuse=False):
    if gconv_type == 'Z2_to_P4':
        h_in = 'Z2'
        h_out = 'C4'
    elif gconv_type == 'Z2_to_P4M':
        h_in = 'Z2'
        h_out = 'D4'
    elif gconv_type == 'P4_to_P4':
        h_in = 'C4'
        h_out = 'C4'
    elif gconv_type == 'P4M_to_P4M':
        h_in = 'D4'
        h_out = 'D4'
    else:
        raise NotImplemented('Unsupported gconv_type: {}!'.format(gconv_type))
    with tf.variable_scope(name, reuse=reuse) as vs:
        gconv_indices, gconv_shape_info, w_shape = gconv2d_util(
                h_input=h_in, h_output=h_out, in_channels=C_in, out_channels=C_out, ksize=ksize)
        w = tf.Variable(tf.truncated_normal(w_shape, stddev=1.))
        x = gconv2d(input=x, filter=w, strides=[1, 1, 1, 1], padding=padding,
                gconv_indices=gconv_indices, gconv_shape_info=gconv_shape_info)
        if bn:
            x = tf.layers.batch_normalization(x, training=is_train)
        return act(x)

def p4m_cnn(x, y_ , is_train, kernel_size=3, stride=1, drop_rate=0.3, reuse=False):
  '''
  x: NHWC
  y_: N
  '''
  C_out = 10
  with tf.variable_scope("P4MCNN", reuse=reuse) as vs:
    x = gconv_bn_act(x, gconv_type='Z2_to_P4M', C_in=1, C_out=C_out, is_train=is_train,
                reuse=reuse, name='Z2_to_P4M_gconv_bn_act1') # l1
    x = tf.layers.dropout(inputs=x, rate=drop_rate,
                 training=is_train, name='drop1')
    print(x.get_shape().as_list()) # [None, 26, 26, 80]

    x = gconv_bn_act(x, gconv_type='P4M_to_P4M', C_in=C_out, C_out=C_out, is_train=is_train,
                reuse=reuse, name='P4M_to_P4M_gconv_bn_act2') # l2
    print(x.get_shape().as_list()) # [None, 24, 24, 80]
    xs = x.get_shape().as_list()
    x = tf.reshape(x, shape=[-1, xs[1], xs[2], 8, xs[3]//8])
    print(x.get_shape().as_list()) # [None, 24, 24, 8, 10]
    x = tf.reduce_max(x, axis=(-2), keep_dims=False, name='reduce_max_l2')
    print(x.get_shape().as_list()) # [None, 24, 24, 10]
    x = tf.layers.max_pooling2d(x, pool_size=2, strides=2,
                    padding='valid',
                    data_format='channels_last', name='max_pool2')
    print(x.get_shape().as_list()) # [None, 12, 12, 10]
    x = gconv_bn_act(x, gconv_type='Z2_to_P4M', C_in=C_out, C_out=C_out, is_train=is_train,
                reuse=reuse, name='Z2_to_P4M_gconv_bn_act3') # l3
    x = tf.layers.dropout(inputs=x, rate=drop_rate,
                training=is_train, name='drop3')
    print(x.get_shape().as_list()) # [None, 10, 10, 80]
    x = gconv_bn_act(x, gconv_type='P4M_to_P4M', C_in=C_out, C_out=C_out, is_train=is_train,
                reuse=reuse, name='P4M_to_P4M_gconv_bn_act4') # l4
    x = tf.layers.dropout(inputs=x, rate=drop_rate,
                training=is_train, name='drop4')
    print(x.get_shape().as_list()) # [None, 8, 8, 80]
    x = gconv_bn_act(x, gconv_type='P4M_to_P4M', C_in=C_out, C_out=C_out, is_train=is_train,
                reuse=reuse, name='P4M_to_P4M_gconv_bn_act5') # l5
    x = tf.layers.dropout(inputs=x, rate=drop_rate,
                training=is_train, name='drop5')
    print(x.get_shape().as_list()) # [None, 6, 6, 80]
    x = gconv_bn_act(x, gconv_type='P4M_to_P4M', C_in=C_out, C_out=C_out, is_train=is_train,
                reuse=reuse, name='P4M_to_P4M_gconv_bn_act6') # l6
    x = tf.layers.dropout(inputs=x, rate=drop_rate,
                training=is_train, name='drop6')
    print(x.get_shape().as_list()) # [None, 4, 4, 80]
    x = gconv_bn_act(x, gconv_type='P4M_to_P4M', C_in=C_out, C_out=C_out, is_train=is_train,
                padding="VALID",
                bn=False,
                act=tf.identity, reuse=reuse, name='P4M_to_P4M_gconv7',) # l7
    print(x.get_shape().as_list()) # [None, 2, 2, 80]

    # take max value of (H,W) dimensions
    # NHWC --> Nx10
    xs = x.get_shape().as_list() 
    x = tf.reshape(x, shape=[-1, xs[1], xs[2], 8, xs[3]//8])
    print(x.get_shape().as_list()) # [None, 2, 2, 8, 10]
    logits = tf.reduce_max(x, axis=(1, 2, 3), keep_dims=False, name='reduce_max_l7')
    print(logits.get_shape().as_list()) # [None, 10]
  return logits


In [3]:
x = tf.placeholder(tf.float32, [None, 28, 28, 1], 'x')
y_ = tf.placeholder(tf.int32, [None], 'y_')
is_train = tf.placeholder(tf.bool, name='is_train')

In [4]:
p4m_cnn(x,y_, True)

[None, 26, 26, 80]
[None, 24, 24, 80]
[None, 24, 24, 8, 10]
[None, 24, 24, 10]
[None, 12, 12, 10]
[None, 10, 10, 80]
[None, 8, 8, 80]
[None, 6, 6, 80]
[None, 4, 4, 80]
[None, 2, 2, 80]
[None, 2, 2, 8, 10]
[None, 10]


<tf.Tensor 'P4MCNN/reduce_max_l7:0' shape=(?, 10) dtype=float32>

In [ ]:

def p4_cnn(x, y_ , is_train, kernel_size=3, stride=1, drop_rate=0.3, reuse=False):
  '''
  x: NHWC
  y_: N
  '''
  C_out = 10
  with tf.variable_scope("P4CNN", reuse=reuse) as vs:
    x = gconv_bn_act(x, gconv_type='Z2_to_P4', C_in=1, C_out=C_out, is_train=is_train,
                reuse=reuse, name='Z2_to_P4_gconv_bn_act1') # l1
    # drop1 = tf.layers.dropout(inputs=conv_bn_act_1, rate=drop_rate,
    #             training=is_train, name='drop1')
    print(x.get_shape().as_list()) # [None, 26, 26, 40]

    x = gconv_bn_act(x, gconv_type='P4_to_P4', C_in=C_out, C_out=C_out, is_train=is_train,
                reuse=reuse, name='P4_to_P4_gconv_bn_act2') # l2
    print(x.get_shape().as_list()) # [None, 24, 24, 40]
    x = tf.layers.max_pooling2d(x, pool_size=2, strides=2,
                    padding='valid',
                    data_format='channels_last', name='max_pool2')
    print(x.get_shape().as_list()) # [None, 12, 12, 40]
    x = gconv_bn_act(x, gconv_type='P4_to_P4', C_in=C_out, C_out=C_out, is_train=is_train,
                reuse=reuse, name='P4_to_P4_gconv_bn_act3') # l3
    # x = tf.layers.dropout(inputs=x, rate=drop_rate,
    #             training=is_train, name='drop3')
    print(x.get_shape().as_list()) # [None, 10, 10, 40]
    x = gconv_bn_act(x, gconv_type='P4_to_P4', C_in=C_out, C_out=C_out, is_train=is_train,
                reuse=reuse, name='P4_to_P4_gconv_bn_act4') # l4
    # x = tf.layers.dropout(inputs=x, rate=drop_rate,
    #             training=is_train, name='drop4')
    print(x.get_shape().as_list()) # [None, 8, 8, 40]
    x = gconv_bn_act(x, gconv_type='P4_to_P4', C_in=C_out, C_out=C_out, is_train=is_train,
                reuse=reuse, name='P4_to_P4_gconv_bn_act5') # l5
    # x = tf.layers.dropout(inputs=x, rate=drop_rate,
    #             training=is_train, name='drop5')
    print(x.get_shape().as_list()) # [None, 6, 6, 40]
    x = gconv_bn_act(x, gconv_type='P4_to_P4', C_in=C_out, C_out=C_out, is_train=is_train,
                reuse=reuse, name='P4_to_P4_gconv_bn_act6') # l6
    # x = tf.layers.dropout(inputs=x, rate=drop_rate,
    #             training=is_train, name='drop6')
    print(x.get_shape().as_list()) # [None, 4, 4, 40]
    x = gconv_bn_act(x, gconv_type='P4_to_P4', C_in=C_out, C_out=C_out, is_train=is_train,
                padding="VALID",
                bn=False,
                act=tf.identity, reuse=reuse, name='P4_to_P4_gconv7',) # l7
    print(x.get_shape().as_list()) # [None, 2, 2, 40]

    # take max value of (H,W) dimensions
    # NHWC --> Nx10
    xs = x.get_shape().as_list() 
    x = tf.reshape(x, shape=[-1, xs[1], xs[2], 4, xs[3]//4])
    print(x.get_shape().as_list()) # [None, 2, 2, 4, 10]
    logits = tf.reduce_max(x, axis=(1, 2, 3), keep_dims=False, name='reduce_max1')
    # print(logits.get_shape().as_list()) # [None, 10]
  return logits

In [ ]:
# Construct graph
x = tf.placeholder(tf.float32, [None, 9, 9, 3])

gconv_indices, gconv_shape_info, w_shape = gconv2d_util(
    h_input='Z2', h_output='D4', in_channels=3, out_channels=64, ksize=3)
w = tf.Variable(tf.truncated_normal(w_shape, stddev=1.))
y = gconv2d(input=x, filter=w, strides=[1, 1, 1, 1], padding='SAME',
            gconv_indices=gconv_indices, gconv_shape_info=gconv_shape_info)
print(y.shape)

gconv_indices, gconv_shape_info, w_shape = gconv2d_util(
    h_input='D4', h_output='D4', in_channels=64, out_channels=64, ksize=3)
w = tf.Variable(tf.truncated_normal(w_shape, stddev=1.))
y = gconv2d(input=y, filter=w, strides=[1, 1, 1, 1], padding='SAME',
            gconv_indices=gconv_indices, gconv_shape_info=gconv_shape_info)
print(y.shape)
# Compute
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
y = sess.run(y, feed_dict={x: np.random.randn(10, 9, 9, 3)})
sess.close()

print(y.shape)  # (10, 9, 9, 512) 

In [3]:
import chainer.functions as F

In [5]:
import chainer.functions as F
from chainer import Chain

class ConvBNAct(Chain):

    def __init__(self,
                 conv,
                 bn=True,
                 act=F.relu):
        super(ConvBNAct, self).__init__(conv=conv)

        if bn:
            out_channels = self.conv.W.data.shape[0]
            self.add_link('bn', F.BatchNormalization(out_channels))
        else:
            self.bn = None

        self.act = act

    def __call__(self, x, train, finetune):

        y = self.conv(x)

        if self.bn:
            y = self.bn(y, test=not train, finetune=finetune)
        if self.act:
            y = self.act(y)

        return y


class Z2CNN(Chain):

    def __init__(self):
        ksize = 3
        bn = True
        act = F.relu
        self.dr = 0.3
        super(Z2CNN, self).__init__(

            l1=ConvBNAct(
                conv=F.Convolution2D(in_channels=1, out_channels=20, ksize=ksize, stride=1, pad=0),
                bn=bn,
                act=act
            ),

            l2=ConvBNAct(
                conv=F.Convolution2D(in_channels=20, out_channels=20, ksize=ksize, stride=1, pad=0),
                bn=bn,
                act=act
            ),

            l3=ConvBNAct(
                conv=F.Convolution2D(in_channels=20, out_channels=20, ksize=ksize, stride=1, pad=0),
                bn=bn,
                act=act
            ),

            l4=ConvBNAct(
                conv=F.Convolution2D(in_channels=20, out_channels=20, ksize=ksize, stride=1, pad=0),
                bn=bn,
                act=act
            ),

            l5=ConvBNAct(
                conv=F.Convolution2D(in_channels=20, out_channels=20, ksize=ksize, stride=1, pad=0),
                bn=bn,
                act=act
            ),

            l6=ConvBNAct(
                conv=F.Convolution2D(in_channels=20, out_channels=20, ksize=ksize, stride=1, pad=0),
                bn=bn,
                act=act
            ),

            top=F.Convolution2D(in_channels=20, out_channels=10, ksize=4, stride=1, pad=0),
        )

    def __call__(self, x, t, train=True, finetune=False):

        h = self.l1(x, train, finetune)
        h = F.dropout(h, self.dr, train)
        h = self.l2(h, train, finetune)

        h = F.max_pooling_2d(h, ksize=2, stride=2, pad=0, cover_all=True, use_cudnn=True)

        h = self.l3(h, train, finetune)
        h = F.dropout(h, self.dr, train)
        h = self.l4(h, train, finetune)
        h = F.dropout(h, self.dr, train)
        h = self.l5(h, train, finetune)
        h = F.dropout(h, self.dr, train)
        h = self.l6(h, train, finetune)
        h = F.dropout(h, self.dr, train)

        h = self.top(h)

        h = F.max(h, axis=-1, keepdims=False)
        h = F.max(h, axis=-1, keepdims=False)

        return F.softmax_cross_entropy(h, t), F.accuracy(h, t)

    def start_finetuning(self):
        for c in self.children():
            if isinstance(c, ConvBNAct):
                if c.bn:
                    c.bn.start_finetuning()



In [8]:
help(os.path.join)

Help on function join in module posixpath:

join(a, *p)
    Join two or more pathname components, inserting '/' as needed.
    If any component is an absolute path, all previous path components
    will be discarded.  An empty last part will result in a path that
    ends with a separator.

